In [1]:
import os
import glob

#from tqdm import tqdm
import numpy as np
import scipy.ndimage
import scipy.misc
import pandas as pd
import pickle
from IPython.display import clear_output

import keras
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers import Concatenate
from keras.layers import Input
from keras.layers.normalization import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Model

from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
# import the necessary packages
from keras.preprocessing import image as image_utils
from keras.utils import plot_model
from keras import backend as K
from keras.regularizers import Regularizer

from keras.wrappers.scikit_learn import KerasRegressor


from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation, grid_search
import keras_metrics

from scipy import stats

import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import random as rn


# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/keras-team/keras/issues/2280#issuecomment-306959926

import os
os.environ['PYTHONHASHSEED'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess) # reference: https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer

Using TensorFlow backend.
/l/anaconda3-5.2.0/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/nfs/nfs5-insecure/home/insecure-ro/software/rhel6_x86_64/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/nfs/nfs5-insecure/home/insecure-ro/software/rhel6_x86_64/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored cl

In [2]:
IMG_SIZE = (256, 256)
IN_SHAPE = (*IMG_SIZE, 3)
BATCH_SIZE = 64

In [3]:
openImg_path = '/nfs/juhu/data/rakhasan/bystander-detection/google-img-db/'
survey_path='/nfs/juhu/data/rakhasan/bystander-detection/pilot-study2/'
survey_photo_path = survey_path+'/photos/'

model_output_path = '/nfs/juhu/data/rakhasan/bystander-detection/code-repos/notebooks/model-output/'

print('loading features.')

'''Note: this data frame has all deep features, and the label is binary values (0,1)'''
feature_df = pickle.load(open(os.path.join(survey_path, 'features-df-all.pkl'), 'rb'))
#feature_df = feature_df[(feature_df.label==1)|(feature_df.label==-1)]
#feature_df['label'] = feature_df.apply(lambda row: 1 if row.label==1 else 0, axis=1)
print('dataset:',len(feature_df), 'unique labels:', feature_df.label.unique(),
     'pos:',len(feature_df[feature_df.label==1]),'neg:',len(feature_df[feature_df.label==0]))

loading features.
dataset: 3802 unique labels: [1 0] pos: 2287 neg: 1515


In [4]:
'''Feature names'''

#joint names labeled by openpose
body_joint_names = ['nose', 'neck', 'Rsho', 'Relb', 'Rwri', 'Lsho', 'Lelb',
               'Lwri', 'Rhip', 'Rkne', 'Rank', 'Lhip', 'Lkne', 'Lank', 
               'Leye', 'Reye', 'Lear', 'Rear']

#angles between pairs of body joint, from openpose
link_angle_features = ['angle_'+str(i) for i in range(17)]

#probability of detecting a body joint, from openpose
body_joint_prob_features = [j + '_prob' for j in body_joint_names]

face_exp_feaures = ['angry', 'disgusted', 'fearful', 'happy', 'sad', 'surprised', 'neutral']

img_level_features = ['person_distance', 'person_size', 'num_people']

visual_features = img_level_features +\
    link_angle_features + body_joint_prob_features + face_exp_feaures

features_from_study = ['was_aware_num',  'posing_num',  'comfort_num',  'will_num', 'photographer_intention_num',
     'replacable_num',  'photo_place_num']

resnet_feature_cols = ['resnet_feat_{}'.format(i) for i in range(131071)]

resnet_feat_avg_cols = ['resnet_feat_avg_{}'.format(i) for i in range(2048)]

all_features = features_from_study + visual_features + resnet_feat_avg_cols

In [47]:
from keras import optimizers

def linear_regression_model(input_dim=38, hidden_dims = []):
    '''
    Create a fully connected network with first layer as input with input_dim=input_dim,
    and len(hidden_dims) number of hidden layers.
    
    Currenly default activation is relu for all hidden layers, and a dropout(.5) is added.
    '''
    
    input_layer = Input(shape=(input_dim,), name = 'input_layer')
    hidden_layer = input_layer
    if hidden_dims:
        for hidden_dim in hidden_dims:
            hidden_layer = Dense(hidden_dim, activation='relu')(hidden_layer)
            hidden_layer = Dropout(.5)(BatchNormalization()(hidden_layer))
    
    output_layer = Dense(1, kernel_regularizer=keras.regularizers.l2(1), bias_regularizer=keras.regularizers.l2(1), 
                         activation = 'linear')(hidden_layer)
    
    model = Model(input_layer, output_layer)
    
    keras.backend.set_epsilon(1) 
    
    model.compile(optimizer=optimizers.SGD( lr = 0.01, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=0.5),
                  loss='mse',
                  metrics=['mse','mae','mape'])
    return model

In [43]:
def do_cross_validation(model_func, X,Y, n_splits=5, epochs=20, save_model = False,
                        save_prefix='', model_args=None, verbose=1):
    seed = 1234
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    cvscores = []
    
    fold = 0
    splits = []
    for train, test in kfold.split(X, (Y>0).astype(int)):
        # create model
        if model_args:
            model = model_func(model_args['input_dim'], model_args['hidden_dims'])
        else:
            model = model_func()
        model.fit(X[train], Y[train], epochs=epochs, batch_size=BATCH_SIZE, verbose=verbose)
        #evaluate the model
        score = model.evaluate(X[test], Y[test], verbose=1)
        print('metrics:{}, values:{}'.format(model.metrics_names, score))
        cvscores.append(score)
        
        if save_model:
            model.save_weights(model_output_path+'model_{}_{}.weights'.format(save_prefix, fold))
            fold+=1
            splits.append((train, test))
    
    if save_model:
        pickle.dump(splits, open(model_output_path+'splits_{}'.format(save_prefix), 'wb'))

    return cvscores

def fine_tune_cv(model_func, feature_df, n_splits=5, epochs=20, save_model = True,
                        save_prefix=''):
    seed = 1234
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    cvscores = []
    
    fold = 0
    splits = []

    for train, test in kfold.split(feature_df, feature_df.label):
        model = model_func()
        
        X_train = np.array([x for x in feature_df.resized_cropped_img.values[train]])
        X_test = np.array([x for x in feature_df.resized_cropped_img.values[test]])

        model.fit(X_train, feature_df.label[train], epochs=epochs, batch_size=BATCH_SIZE, verbose=1)
        
        #evaluate the model
        scores = model.evaluate(X_test, feature_df.label[test], verbose=1)
        print(model.metrics_names, scores)
        cvscores.append(scores)
        
        if save_model:
            model.save_weights(model_output_path+'model_{}_{}.weights'.format(save_prefix, fold))
            fold+=1
            splits.append((train, test))
    
    if save_model:
        pickle.dump(splits, open(model_output_path+'splits_{}'.format(save_prefix), 'wb'))

    return cvscores


def fine_tune_cv_mixed_features(model_func, feature_df, other_feat, hidden_dims=[],
                                n_splits=10, epochs=20, save_model = True, save_prefix = ''):
    seed = 1234
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    cvscores = []
    
    fold = 0
    splits = []
    for train, test in kfold.split(feature_df, feature_df.label):
        
        X_train = np.array([x for x in feature_df.resized_cropped_img.values[train]])
        X_test = np.array([x for x in feature_df.resized_cropped_img.values[test]])

        model = resnet_mixed_features(len(other_feat), hidden_dims)
        
        model.fit([X_train, feature_df[other_feat].apply(stats.zscore).values[train]],
                  feature_df.label[train], epochs=epochs, batch_size=BATCH_SIZE, verbose=1)
        
        
        scores = model.evaluate([X_test, feature_df[other_feat].apply(stats.zscore).values[test]],
                                feature_df.label[test], verbose=1)
        print(model.metrics_names, scores)
        cvscores.append(scores)
        
        if save_model:
            model.save_weights(model_output_path+'model_{}_{}.weights'.format(save_prefix,fold))
            fold+=1
            splits.append((train, test))
    
    if save_model:
        pickle.dump(splits, open(model_output_path+'splits_{}'.format(save_prefix), 'wb'))
        
    return cvscores

## Predict pose

### Built in LinearRegression model

In [14]:
'''Built in normalization, which performs a z-score transformation'''

feat = img_level_features
print('Features:',feat)
model = LinearRegression(fit_intercept=True, normalize=True, copy_X=True)
scores = cross_val_score(model, feature_df[feat], feature_df.posing_num, cv=5, scoring='neg_mean_squared_error')
print("score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print()

feat = img_level_features + link_angle_features + body_joint_prob_features
print('Features:',feat)
model = LinearRegression(fit_intercept=True, normalize=True, copy_X=True)
scores = cross_val_score(model, feature_df[feat], feature_df.posing_num, cv=5, scoring='neg_mean_squared_error')
print("score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print()

feat = img_level_features + link_angle_features + body_joint_prob_features + face_exp_feaures
print('Features:',feat)
model = LinearRegression(fit_intercept=True, normalize=True, copy_X=True)
scores = cross_val_score(model, feature_df[feat], feature_df.posing_num, cv=5, scoring='neg_mean_squared_error')
print("score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print()

'''use all features'''
feat = resnet_feat_avg_cols + img_level_features + link_angle_features + body_joint_prob_features + face_exp_feaures
print('Features: all features')
model = LinearRegression(fit_intercept=True, normalize=True, copy_X=True)
scores = cross_val_score(model, feature_df[feat], feature_df.posing_num, cv=5, scoring='neg_mean_squared_error')
print("score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print()

Features: ['person_distance', 'person_size', 'num_people']
score: -2.12 (+/- 0.34)

Features: ['person_distance', 'person_size', 'num_people', 'angle_0', 'angle_1', 'angle_2', 'angle_3', 'angle_4', 'angle_5', 'angle_6', 'angle_7', 'angle_8', 'angle_9', 'angle_10', 'angle_11', 'angle_12', 'angle_13', 'angle_14', 'angle_15', 'angle_16', 'nose_prob', 'neck_prob', 'Rsho_prob', 'Relb_prob', 'Rwri_prob', 'Lsho_prob', 'Lelb_prob', 'Lwri_prob', 'Rhip_prob', 'Rkne_prob', 'Rank_prob', 'Lhip_prob', 'Lkne_prob', 'Lank_prob', 'Leye_prob', 'Reye_prob', 'Lear_prob', 'Rear_prob']
score: -1.81 (+/- 0.30)

Features: ['person_distance', 'person_size', 'num_people', 'angle_0', 'angle_1', 'angle_2', 'angle_3', 'angle_4', 'angle_5', 'angle_6', 'angle_7', 'angle_8', 'angle_9', 'angle_10', 'angle_11', 'angle_12', 'angle_13', 'angle_14', 'angle_15', 'angle_16', 'nose_prob', 'neck_prob', 'Rsho_prob', 'Relb_prob', 'Rwri_prob', 'Lsho_prob', 'Lelb_prob', 'Lwri_prob', 'Rhip_prob', 'Rkne_prob', 'Rank_prob', 'Lhip_pr

### Keras Linear Regression model

### Test *posing* with different feature sets and network layers

In [12]:
len(img_level_features + link_angle_features + body_joint_prob_features)

38

In [13]:
'''use img_level_features'''
feat = img_level_features
print('Features:',feat)

scores = do_cross_validation(linear_regression_model, 
                            X= feature_df[feat].apply(stats.zscore).values,
                            Y= feature_df.posing_num, epochs= 20, n_splits = 5,
                            model_args={'input_dim': len(feat),'hidden_dims':[]})

metrics=['loss','mse','mae']

print()
for m in range(len(metrics)):
    print("{}:  {:.2f} (+/- {:.2f})".format(metrics[m], np.mean([s[m] for s in scores]), np.std([s[m] for s in scores]) * 2))
print()

Features: ['person_distance', 'person_size', 'num_people']
Epoch 1/20
3041/3041 [==============================] - 0s 87us/step - loss: 6.0132 - mean_squared_error: 5.1143 - mean_absolute_error: 1.8766
Epoch 2/20
3041/3041 [==============================] - 0s 12us/step - loss: 2.6044 - mean_squared_error: 2.3078 - mean_absolute_error: 1.2763
Epoch 3/20
3041/3041 [==============================] - 0s 12us/step - loss: 2.5919 - mean_squared_error: 2.3318 - mean_absolute_error: 1.2871
Epoch 4/20
3041/3041 [==============================] - 0s 12us/step - loss: 2.5978 - mean_squared_error: 2.3230 - mean_absolute_error: 1.2829
Epoch 5/20
3041/3041 [==============================] - 0s 12us/step - loss: 2.5949 - mean_squared_error: 2.3315 - mean_absolute_error: 1.2865
Epoch 6/20
3041/3041 [==============================] - 0s 13us/step - loss: 2.5904 - mean_squared_error: 2.3195 - mean_absolute_error: 1.2824
Epoch 7/20
3041/3041 [==============================] - 0s 13us/step - loss: 2.5925

In [14]:
feat = img_level_features + link_angle_features +body_joint_prob_features #+ resnet_feat_avg_cols
scores = do_cross_validation(linear_regression_model, X= feature_df[feat].apply(stats.zscore).values,
                            Y= feature_df.posing_num, epochs= 30, n_splits = 10,
                            model_args={'input_dim': len(feat),'hidden_dims':[]})

metrics=['loss','mse','mae']

print()
for m in range(len(metrics)):
    print("{}:  {:.2f} (+/- {:.2f})".format(metrics[m], np.mean([s[m] for s in scores]), np.std([s[m] for s in scores]) * 2))
print()

Epoch 1/30
3421/3421 [==============================] - 0s 40us/step - loss: 4.2968 - mean_squared_error: 3.3011 - mean_absolute_error: 1.5037
Epoch 2/30
3421/3421 [==============================] - 0s 13us/step - loss: 2.3372 - mean_squared_error: 2.0305 - mean_absolute_error: 1.2007
Epoch 3/30
3421/3421 [==============================] - 0s 13us/step - loss: 2.3301 - mean_squared_error: 2.0295 - mean_absolute_error: 1.2009
Epoch 4/30
3421/3421 [==============================] - 0s 13us/step - loss: 2.3327 - mean_squared_error: 2.0312 - mean_absolute_error: 1.2034
Epoch 5/30
3421/3421 [==============================] - 0s 13us/step - loss: 2.3395 - mean_squared_error: 2.0406 - mean_absolute_error: 1.2065
Epoch 6/30
3421/3421 [==============================] - 0s 13us/step - loss: 2.3373 - mean_squared_error: 2.0390 - mean_absolute_error: 1.2065
Epoch 7/30
3421/3421 [==============================] - 0s 13us/step - loss: 2.3303 - mean_squared_error: 2.0152 - mean_absolute_error: 1.1958

3422/3422 [==============================] - 0s 13us/step - loss: 2.3246 - mean_squared_error: 2.0335 - mean_absolute_error: 1.2019
Epoch 21/30
3422/3422 [==============================] - 0s 13us/step - loss: 2.3222 - mean_squared_error: 2.0195 - mean_absolute_error: 1.1974
Epoch 22/30
3422/3422 [==============================] - 0s 13us/step - loss: 2.3302 - mean_squared_error: 2.0245 - mean_absolute_error: 1.1961
Epoch 23/30
3422/3422 [==============================] - 0s 13us/step - loss: 2.3278 - mean_squared_error: 2.0338 - mean_absolute_error: 1.2000
Epoch 24/30
3422/3422 [==============================] - 0s 13us/step - loss: 2.3222 - mean_squared_error: 2.0309 - mean_absolute_error: 1.2018
Epoch 25/30
3422/3422 [==============================] - 0s 13us/step - loss: 2.3231 - mean_squared_error: 2.0153 - mean_absolute_error: 1.1964
Epoch 26/30
3422/3422 [==============================] - 0s 13us/step - loss: 2.3233 - mean_squared_error: 2.0264 - mean_absolute_error: 1.1994
Epoc

3422/3422 [==============================] - 0s 14us/step - loss: 2.3343 - mean_squared_error: 2.0338 - mean_absolute_error: 1.2006
Epoch 9/30
3422/3422 [==============================] - 0s 14us/step - loss: 2.3365 - mean_squared_error: 2.0370 - mean_absolute_error: 1.2020
Epoch 10/30
3422/3422 [==============================] - 0s 14us/step - loss: 2.3397 - mean_squared_error: 2.0340 - mean_absolute_error: 1.2014
Epoch 11/30
3422/3422 [==============================] - 0s 14us/step - loss: 2.3338 - mean_squared_error: 2.0391 - mean_absolute_error: 1.2027
Epoch 12/30
3422/3422 [==============================] - 0s 14us/step - loss: 2.3409 - mean_squared_error: 2.0419 - mean_absolute_error: 1.2045
Epoch 13/30
3422/3422 [==============================] - 0s 14us/step - loss: 2.3345 - mean_squared_error: 2.0269 - mean_absolute_error: 1.1975
Epoch 14/30
3422/3422 [==============================] - 0s 14us/step - loss: 2.3445 - mean_squared_error: 2.0489 - mean_absolute_error: 1.2062
Epoch

In [15]:
feat = resnet_feat_avg_cols
scores = do_cross_validation(linear_regression_model, X= feature_df[feat].apply(stats.zscore).values,
                            Y= feature_df.posing_num, epochs= 30, n_splits = 10,
                            model_args={'input_dim': len(feat),'hidden_dims':[]})

metrics=['loss','mse','mae']

print()
for m in range(len(metrics)):
    print("{}:  {:.2f} (+/- {:.2f})".format(metrics[m], np.mean([s[m] for s in scores]), np.std([s[m] for s in scores]) * 2))
print()

Epoch 1/30
3421/3421 [==============================] - 0s 79us/step - loss: 5.0720 - mean_squared_error: 3.5261 - mean_absolute_error: 1.5311
Epoch 2/30
3421/3421 [==============================] - 0s 21us/step - loss: 3.1216 - mean_squared_error: 2.2796 - mean_absolute_error: 1.2399
Epoch 3/30
3421/3421 [==============================] - 0s 21us/step - loss: 2.4583 - mean_squared_error: 1.9270 - mean_absolute_error: 1.1453
Epoch 4/30
3421/3421 [==============================] - 0s 21us/step - loss: 2.1799 - mean_squared_error: 1.7729 - mean_absolute_error: 1.0990
Epoch 5/30
3421/3421 [==============================] - 0s 21us/step - loss: 2.0931 - mean_squared_error: 1.7195 - mean_absolute_error: 1.0816
Epoch 6/30
3421/3421 [==============================] - 0s 21us/step - loss: 2.0461 - mean_squared_error: 1.6884 - mean_absolute_error: 1.0697
Epoch 7/30
3421/3421 [==============================] - 0s 22us/step - loss: 2.0227 - mean_squared_error: 1.6700 - mean_absolute_error: 1.0604

3422/3422 [==============================] - 0s 21us/step - loss: 1.9798 - mean_squared_error: 1.6283 - mean_absolute_error: 1.0498
Epoch 21/30
3422/3422 [==============================] - 0s 21us/step - loss: 1.9876 - mean_squared_error: 1.6371 - mean_absolute_error: 1.0517
Epoch 22/30
3422/3422 [==============================] - 0s 22us/step - loss: 1.9815 - mean_squared_error: 1.6281 - mean_absolute_error: 1.0485
Epoch 23/30
3422/3422 [==============================] - 0s 21us/step - loss: 1.9957 - mean_squared_error: 1.6381 - mean_absolute_error: 1.0520
Epoch 24/30
3422/3422 [==============================] - 0s 21us/step - loss: 1.9857 - mean_squared_error: 1.6324 - mean_absolute_error: 1.0535
Epoch 25/30
3422/3422 [==============================] - 0s 21us/step - loss: 1.9748 - mean_squared_error: 1.6146 - mean_absolute_error: 1.0468
Epoch 26/30
3422/3422 [==============================] - 0s 22us/step - loss: 1.9927 - mean_squared_error: 1.6359 - mean_absolute_error: 1.0505
Epoc

3422/3422 [==============================] - 0s 22us/step - loss: 2.0176 - mean_squared_error: 1.6583 - mean_absolute_error: 1.0598
Epoch 9/30
3422/3422 [==============================] - 0s 22us/step - loss: 2.0302 - mean_squared_error: 1.6764 - mean_absolute_error: 1.0659
Epoch 10/30
3422/3422 [==============================] - 0s 22us/step - loss: 2.0206 - mean_squared_error: 1.6635 - mean_absolute_error: 1.0636
Epoch 11/30
3422/3422 [==============================] - 0s 22us/step - loss: 2.0186 - mean_squared_error: 1.6632 - mean_absolute_error: 1.0632
Epoch 12/30
3422/3422 [==============================] - 0s 22us/step - loss: 2.0052 - mean_squared_error: 1.6457 - mean_absolute_error: 1.0556
Epoch 13/30
3422/3422 [==============================] - 0s 22us/step - loss: 2.0238 - mean_squared_error: 1.6668 - mean_absolute_error: 1.0584
Epoch 14/30
3422/3422 [==============================] - 0s 22us/step - loss: 2.0183 - mean_squared_error: 1.6597 - mean_absolute_error: 1.0612
Epoch

#### Add a hidden layer

In [16]:
'''Only resnet feature, no normalization'''
feat = resnet_feat_avg_cols
scores = do_cross_validation(linear_regression_model, X= feature_df[feat].values,#.apply(stats.zscore).values,
                            Y= feature_df.posing_num, epochs= 30, n_splits = 10,
                            model_args={'input_dim': len(feat),'hidden_dims':[64]})

metrics=['loss','mse','mae']

print()
for m in range(len(metrics)):
    print("{}:  {:.2f} (+/- {:.2f})".format(metrics[m], np.mean([s[m] for s in scores]), np.std([s[m] for s in scores]) * 2))
print()

Epoch 1/30
3421/3421 [==============================] - 1s 196us/step - loss: 7.2043 - mean_squared_error: 5.3276 - mean_absolute_error: 1.8577
Epoch 2/30
3421/3421 [==============================] - 0s 69us/step - loss: 4.1800 - mean_squared_error: 3.1512 - mean_absolute_error: 1.4382
Epoch 3/30
3421/3421 [==============================] - 0s 69us/step - loss: 2.6866 - mean_squared_error: 2.2845 - mean_absolute_error: 1.2573
Epoch 4/30
3421/3421 [==============================] - 0s 72us/step - loss: 2.2689 - mean_squared_error: 2.0470 - mean_absolute_error: 1.1858
Epoch 5/30
3421/3421 [==============================] - 0s 68us/step - loss: 2.1500 - mean_squared_error: 1.9496 - mean_absolute_error: 1.1524
Epoch 6/30
3421/3421 [==============================] - 0s 68us/step - loss: 2.0734 - mean_squared_error: 1.8761 - mean_absolute_error: 1.1260
Epoch 7/30
3421/3421 [==============================] - 0s 74us/step - loss: 2.0358 - mean_squared_error: 1.8450 - mean_absolute_error: 1.116

3422/3422 [==============================] - 0s 70us/step - loss: 1.5629 - mean_squared_error: 1.3935 - mean_absolute_error: 0.9535
Epoch 21/30
3422/3422 [==============================] - 0s 73us/step - loss: 1.5542 - mean_squared_error: 1.3782 - mean_absolute_error: 0.9471
Epoch 22/30
3422/3422 [==============================] - 0s 70us/step - loss: 1.4872 - mean_squared_error: 1.3099 - mean_absolute_error: 0.9183
Epoch 23/30
3422/3422 [==============================] - 0s 72us/step - loss: 1.4463 - mean_squared_error: 1.2750 - mean_absolute_error: 0.9109
Epoch 24/30
3422/3422 [==============================] - 0s 73us/step - loss: 1.4654 - mean_squared_error: 1.2995 - mean_absolute_error: 0.9193
Epoch 25/30
3422/3422 [==============================] - 0s 70us/step - loss: 1.4204 - mean_squared_error: 1.2556 - mean_absolute_error: 0.9012
Epoch 26/30
3422/3422 [==============================] - 0s 70us/step - loss: 1.3757 - mean_squared_error: 1.2025 - mean_absolute_error: 0.8780
Epoc

3422/3422 [==============================] - 0s 71us/step - loss: 1.9972 - mean_squared_error: 1.8136 - mean_absolute_error: 1.1057
Epoch 9/30
3422/3422 [==============================] - 0s 71us/step - loss: 1.9503 - mean_squared_error: 1.7693 - mean_absolute_error: 1.0889
Epoch 10/30
3422/3422 [==============================] - 0s 70us/step - loss: 1.8833 - mean_squared_error: 1.7054 - mean_absolute_error: 1.0687
Epoch 11/30
3422/3422 [==============================] - 0s 72us/step - loss: 1.8893 - mean_squared_error: 1.7139 - mean_absolute_error: 1.0628
Epoch 12/30
3422/3422 [==============================] - 0s 70us/step - loss: 1.8067 - mean_squared_error: 1.6303 - mean_absolute_error: 1.0426
Epoch 13/30
3422/3422 [==============================] - 0s 70us/step - loss: 1.8212 - mean_squared_error: 1.6483 - mean_absolute_error: 1.0442
Epoch 14/30
3422/3422 [==============================] - 0s 73us/step - loss: 1.7435 - mean_squared_error: 1.5750 - mean_absolute_error: 1.0220
Epoch

In [28]:
'''All features, normalized, this gives the BEST result'''
min_max_scaler = preprocessing.MinMaxScaler()

feat = resnet_feat_avg_cols + link_angle_features + body_joint_prob_features + face_exp_feaures
scores = do_cross_validation(linear_regression_model, 
                             X= min_max_scaler.fit_transform(feature_df[feat]),
                            Y= feature_df.posing_num, epochs= 30, n_splits = 10,
                            model_args={'input_dim': len(feat),'hidden_dims':[64]})

metrics=['loss','mse','mae']

print()
for m in range(len(metrics)):
    print("{}:  {:.2f} (+/- {:.2f})".format(metrics[m], np.mean([s[m] for s in scores]), np.std([s[m] for s in scores]) * 2))
print()

Epoch 1/30
3421/3421 [==============================] - 3s 841us/step - loss: 5.6781 - mean_squared_error: 4.1827 - mean_absolute_error: 1.6397
Epoch 2/30
3421/3421 [==============================] - 0s 75us/step - loss: 3.0843 - mean_squared_error: 2.3856 - mean_absolute_error: 1.2747
Epoch 3/30
3421/3421 [==============================] - 0s 75us/step - loss: 2.2717 - mean_squared_error: 1.9801 - mean_absolute_error: 1.1707
Epoch 4/30
3421/3421 [==============================] - 0s 76us/step - loss: 2.0689 - mean_squared_error: 1.8468 - mean_absolute_error: 1.1267
Epoch 5/30
3421/3421 [==============================] - 0s 76us/step - loss: 1.9774 - mean_squared_error: 1.7753 - mean_absolute_error: 1.1023
Epoch 6/30
3421/3421 [==============================] - 0s 76us/step - loss: 1.8942 - mean_squared_error: 1.6791 - mean_absolute_error: 1.0628
Epoch 7/30
3421/3421 [==============================] - 0s 76us/step - loss: 1.8275 - mean_squared_error: 1.6276 - mean_absolute_error: 1.042

3422/3422 [==============================] - 0s 78us/step - loss: 1.3051 - mean_squared_error: 1.1189 - mean_absolute_error: 0.8499
Epoch 21/30
3422/3422 [==============================] - 0s 78us/step - loss: 1.2925 - mean_squared_error: 1.1031 - mean_absolute_error: 0.8521
Epoch 22/30
3422/3422 [==============================] - 0s 78us/step - loss: 1.2568 - mean_squared_error: 1.0637 - mean_absolute_error: 0.8271
Epoch 23/30
3422/3422 [==============================] - 0s 77us/step - loss: 1.2346 - mean_squared_error: 1.0497 - mean_absolute_error: 0.8184
Epoch 24/30
3422/3422 [==============================] - 0s 76us/step - loss: 1.2198 - mean_squared_error: 1.0226 - mean_absolute_error: 0.8144
Epoch 25/30
3422/3422 [==============================] - 0s 76us/step - loss: 1.2254 - mean_squared_error: 1.0386 - mean_absolute_error: 0.8160
Epoch 26/30
3422/3422 [==============================] - 0s 76us/step - loss: 1.1954 - mean_squared_error: 1.0060 - mean_absolute_error: 0.8028
Epoc

3422/3422 [==============================] - 0s 76us/step - loss: 1.8001 - mean_squared_error: 1.6041 - mean_absolute_error: 1.0390
Epoch 9/30
3422/3422 [==============================] - 0s 77us/step - loss: 1.7617 - mean_squared_error: 1.5677 - mean_absolute_error: 1.0219
Epoch 10/30
3422/3422 [==============================] - 0s 76us/step - loss: 1.7148 - mean_squared_error: 1.5351 - mean_absolute_error: 1.0076
Epoch 11/30
3422/3422 [==============================] - 0s 76us/step - loss: 1.6551 - mean_squared_error: 1.4721 - mean_absolute_error: 0.9868
Epoch 12/30
3422/3422 [==============================] - 0s 77us/step - loss: 1.6233 - mean_squared_error: 1.4380 - mean_absolute_error: 0.9723
Epoch 13/30
3422/3422 [==============================] - 0s 76us/step - loss: 1.5586 - mean_squared_error: 1.3730 - mean_absolute_error: 0.9475
Epoch 14/30
3422/3422 [==============================] - 0s 76us/step - loss: 1.5588 - mean_squared_error: 1.3761 - mean_absolute_error: 0.9494
Epoch

### Test how well other high level features can be predicted using the low level predictors that best worked for *posing*

In [48]:
feat = resnet_feat_avg_cols + link_angle_features + body_joint_prob_features + face_exp_feaures #+ img_level_features

high_level_concepts = ['was_aware', 'photographer_intention', 'replacable',  'posing','comfort', 'will']
high_level_concepts_num = [c+'_num' for c in high_level_concepts]
metrics=['loss','mse','mae','mape','mase']
all_scores=[]
X= min_max_scaler.fit_transform(feature_df[feat])
for dv in high_level_concepts_num:
    scores = do_cross_validation(linear_regression_model, X,
                            Y= feature_df[dv], epochs= 30, n_splits = 10,
                            model_args={'input_dim': len(feat),'hidden_dims':[64]}, verbose=0)
    all_scores.append(scores)

    print('predicting :',dv)
    for m in range(len(metrics)):
        print("{}:  {:.2f} (+/- {:.2f})".format(metrics[m], np.mean([s[m] for s in scores]), np.std([s[m] for s in scores]) * 2))
    print()
    
tab = dict()
for i in range(len(high_level_concepts_num)):
    print('predicting :',high_level_concepts_num[i])
    for m in range(len(metrics)):
        scores= all_scores[i]
        print("{}:  {:.2f} (+/- {:.2f})".format(
            metrics[m], np.mean([s[m] for s in scores]), np.std([s[m] for s in scores]) * 2))
        #tab[high_level_concepts_num[i]][]
    print()
    

381/381 [==============================] - 2s 5ms/step
metrics:['loss', 'mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'], values:[1.7488865520697565, 1.5924175937031824, 1.0352144394646792, 67.15474572769926]
381/381 [==============================] - 2s 5ms/step
metrics:['loss', 'mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'], values:[1.6716031533839508, 1.5307797891261383, 0.9912607846610503, 64.58514970994683]
381/381 [==============================] - 2s 5ms/step
metrics:['loss', 'mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'], values:[1.7212156032326966, 1.568381793855682, 0.9855427069301055, 65.28281192328986]
380/380 [==============================] - 2s 5ms/step
metrics:['loss', 'mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'], values:[1.797455334663391, 1.6536383666490253, 1.0315145222764266, 65.60587744461863]
380/380 [==============================] - 2s 

IndexError: list index out of range

In [26]:

high_level_concepts_name = {'was_aware_num':'Awareness', 'posing_num':'Pose',
                            'comfort_num':'Comfort', 'will_num':'Willingness', 
                            'photographer_intention_num':'Photographer intention',
                            'replacable_num':'Replaceable', 'photo_place_num':'Photo place'}

In [28]:
'''create table for paper'''    
tab = []
for i in range(len(high_level_concepts_num)):
    d=dict()
    
    for m in range(len(metrics)):
        scores= all_scores[i]
        d['name']=high_level_concepts_name[high_level_concepts_num[i]]
        d[metrics[m]] = np.mean([s[m] for s in scores])
        d[metrics[m]+'-std']=np.std([s[m] for s in scores])
    tab.append(d)
print(pd.DataFrame(tab).set_index('name').round(2).to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &  loss &  loss-std &   mae &  mae-std &   mse &  mse-std \\
name                   &       &           &       &          &       &          \\
\midrule
Awareness              &  1.79 &      0.07 &  1.04 &     0.02 &  1.65 &     0.06 \\
Photographer intention &  2.65 &      0.15 &  1.30 &     0.04 &  2.47 &     0.15 \\
Replacable             &  1.60 &      0.08 &  0.98 &     0.03 &  1.46 &     0.07 \\
Pose                   &  1.99 &      0.14 &  1.08 &     0.05 &  1.81 &     0.14 \\
Comfort                &  0.81 &      0.05 &  0.67 &     0.03 &  0.72 &     0.05 \\
Willingness            &  0.45 &      0.02 &  0.50 &     0.02 &  0.40 &     0.02 \\
\bottomrule
\end{tabular}



## Predict 'Replaceable'


In [13]:
feat = resnet_feat_avg_cols + img_level_features + link_angle_features + body_joint_prob_features + face_exp_feaures
scores = do_cross_validation(linear_regression_model, X= feature_df[feat].apply(stats.zscore).values,
                            Y= feature_df.replacable_num, epochs= 30, n_splits = 10,
                            model_args={'input_dim': len(feat),'hidden_dims':[64]}, verbose=0)

metrics=['loss','mse','mae']

print()
for m in range(len(metrics)):
    print("{}:  {:.2f} (+/- {:.2f})".format(metrics[m], np.mean([s[m] for s in scores]), np.std([s[m] for s in scores]) * 2))
print()

381/381 [==============================] - 0s 99us/step
metrics:['loss', 'mean_squared_error', 'mean_absolute_error'], values:[1.4937870173942385, 1.3707459881862631, 0.9343677277327209]
381/381 [==============================] - 0s 136us/step
metrics:['loss', 'mean_squared_error', 'mean_absolute_error'], values:[1.503943408255189, 1.3729439423153094, 0.9324095655926882]
381/381 [==============================] - 0s 167us/step
metrics:['loss', 'mean_squared_error', 'mean_absolute_error'], values:[1.4647528674658827, 1.328376243120729, 0.9397835080704977]
381/381 [==============================] - 0s 193us/step
metrics:['loss', 'mean_squared_error', 'mean_absolute_error'], values:[1.4751834318706682, 1.3381649897167376, 0.9337431899518791]
381/381 [==============================] - 0s 223us/step
metrics:['loss', 'mean_squared_error', 'mean_absolute_error'], values:[1.5740005732208413, 1.4416623869593062, 0.9821044650916352]
381/381 [==============================] - 0s 234us/step
metric

KeyboardInterrupt: 